# Семинар 2

В рамках сегодняшнего занятия мы посмотрим как можно использовать передовые современные LLM, как можно бороться с галлюционированием языковых моделей и как обучать эти тяжелые модели при  ограниченных ресурсах 

In [1]:
# !pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/accelerate.git

In [1]:
import os
import sys
import random
from typing import List

from tqdm.auto import tqdm, trange
from nltk import sent_tokenize
from sklearn.utils import shuffle
import pandas as pd
import numpy as np
import torch
from datasets import load_dataset
import transformers
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import faiss

Квантование в 4-битную точность с помощью QLoRA позволяет эффективно обучать современные LLM на потребительском оборудовании, сохраняя при этом высокую производительность.

QLoRA квантует предобученную языковую модель до 4 бит и замораживает параметры. Затем к модели добавляется небольшое количество обучаемых слоев Low-Rank Adapter.

Во время обучения градиенты протекают через замороженную квантованную в 4 бита модель и считаются только в LoRA слоях. Таким образом, вся преобученная модель остается без изменений, только квантуется в 4-битную точность, в то время как обновляются только адаптеры. Кроме того, квантование в 4 бита не ухудшает производительность модели.

![](./images/qlora.png)

In [2]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

### inference инструктивных моделей

Давайте попробуем посмотреть, как можно загружать предобученные модели и работать с ними

In [4]:
# model_id = "mistralai/Mistral-7B-v0.1"
model_id = "TheBloke/Llama-2-7B-Chat-fp16"

llm_model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
llm_tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/snv/Projects/courses/SF_generation_NLP/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/snv/Projects/courses/SF_generation_NLP/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.wa

In [46]:
def get_completion(query: str, model, tokenizer) -> str:
  device = "cuda"

  prompt_template = """
  Ниже инструкция, которая описывает задачу. Напиши подробный ответ.
  ### Вопрос:
  {query}

  ### Ответ:
  """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)

  generated_ids = model.generate(**model_inputs, max_new_tokens=200, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
  return (decoded[0])

Попробуем сгенерировать ответ на стандартный вопрос, и посмотрим как отвечает модель. Если вы попробуете запустить этот запрос, но на английском языке, то модель лучше справится с этим запросом. 

In [47]:
result = get_completion(query="столица России", model=llm_model, tokenizer=llm_tokenizer)
print(result)


  Ниже инструкция, которая описывает задачу. Напиши подробный ответ.
  ### Вопрос:
  столица России
  
    ### Релеватный контекст:
    
    
 ### Ответ:
Москва является столицей России с 1703 года, когда Пётр I перенёс столицу из Москвы в Санкт-Петербург. Однако, до этого времени Moscow была столицей России в течение нескольких столетий.

Москва была основана в 1147 году, и в течение нескольких столетий она была столицей Кыргызской и Торгутской землей. В 1238 году, после завоевания Золотой Орды, Москва стала столицей Золотой Орды. В 1322 году, после распада Золотой Орды, Москва стала столицей Галицко-Русского государства.
В 1480-х годах, после упразднения Галицко-Русского государства, Москва стала столицей Великого княжества Литовского. В 1547 году, после разрушения Москвы татарами, она стала столицей Русского государства.
В 1703 году, после переноса столицы в Санкт-Петербург, Москва утратила статус столицы России. Однако, она сохранила свое значение как исторический и культурный цен

Как видите, модель начинает галлюционировать по мере продолжения генерации

### RAG-prepare text

Попробуем полечить это добавлением контекста содержащего полезную информацию для модели

In [4]:
# !pip install apache_beam mwparserfromhell

Загрузим данные из русской Википедии и подготовим индекс, по которому будем искать релевантную  информацию

In [3]:
wiki_data = load_dataset("wikimedia/wikipedia", "20231101.ru")

Resolving data files:   0%|          | 0/21 [00:00<?, ?it/s]

In [4]:
wiki_data['train'][23]

{'id': '42',
 'url': 'https://ru.wikipedia.org/wiki/%D0%A1%D0%BE%D0%B2%D0%B5%D1%82%D1%81%D0%BA%D0%BE-%D0%B3%D0%B5%D1%80%D0%BC%D0%B0%D0%BD%D1%81%D0%BA%D0%B8%D0%B5%20%D1%81%D0%BE%D0%B3%D0%BB%D0%B0%D1%88%D0%B5%D0%BD%D0%B8%D1%8F%20%281939%E2%80%941941%29',
 'title': 'Советско-германские соглашения (1939—1941)',
 'text': 'Советско-германские договоры и соглашения 1939—1941\xa0годов, и их дополнения:\n\n 19 августа 1939\xa0— Германо-советское торговое соглашение.\n Германия предоставила СССР кредит на 200\xa0млн германских марок и взяла на себя обязательство поставить Советскому Союзу по этому кредиту станки и другое заводское оборудование, а также военную технику; СССР обязался погасить кредит поставками сырья и продовольствия.\n 11 февраля 1940\xa0— Хозяйственное соглашение между Германией и СССР о расширении торговли.\n 10 января 1941\xa0— Соглашение о взаимных торговых поставках до августа 1942\xa0года.\n Серьёзное экономическое и военно-техническое сотрудничество между Германией и СССР 

Чтобы не строить индекс для всей Википедии (это долго и вычислительно затратно) найдем статьи, содержащие слово `Москва` в заголовке и после этого добавим одну эту статью к финальной случайной подвыборке 

In [5]:
needed_ids = []

for ind, article in enumerate(tqdm(wiki_data['train'])):
    if 'москва' in article['title'].lower():
        needed_ids.append((ind, article['id']))
len(needed_ids)

  0%|          | 0/1945063 [00:00<?, ?it/s]

2433

In [8]:
needed_ids[:10]

[(50, '71'),
 (2039, '4292'),
 (4359, '7781'),
 (4361, '7783'),
 (6286, '13495'),
 (7306, '15925'),
 (10909, '25701'),
 (10916, '25714'),
 (12652, '32759'),
 (13459, '35192')]

In [9]:
wiki_data['train'][50]

{'id': '71',
 'url': 'https://ru.wikipedia.org/wiki/%D0%9C%D0%BE%D1%81%D0%BA%D0%B2%D0%B0',
 'title': 'Москва',
 'text': 'Москва́ ()\xa0— столица России, город федерального значения, административный центр Центрального федерального округа и центр Московской области, в состав которой не входит. Мегаполис; крупнейший по численности населения город России и её субъект\xa0—  человек (), самый населённый из городов, полностью расположенных в Европе, занимает 22-е место среди городов мира по численности населения. Центр Московской городской агломерации. Самый крупный город Европы по площади.\n\nАрхеологические данные свидетельствуют о существовании городского поселения в районе будущего Кремля с XI века. Первое письменное упоминание о Москве относится к 1147 году: история города насчитывает, по меньшей мере, . Москва\xa0— историческая столица Великого княжества Московского, Русского царства, Российской империи (в 1728—1732\xa0годах), Советской России и СССР. Город-герой. В Москве находятся фе

Берем с 51 документа, только для того, чтобы точно не попала статья про Москву и не забываем добавить найденную потенциально полезную статью

In [4]:
N_ARTICLES_FOR_SUBSAMPLE = 10_000

wiki_subsample = random.sample(wiki_data['train'][51:]['text'], k=N_ARTICLES_FOR_SUBSAMPLE - 1)
wiki_subsample.append(wiki_data['train'][50]['text'])

подготовим код для препроцессинга данных - удаление ударений, неразрывных пробелов и пустых  скобок

In [5]:
import unicodedata

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

In [6]:
def replace_characters(original, new, text_str):
    for i in range(len(original) - 1):
        text_str = text_str.replace(original[i], new[i])
    return text_str

#буквы, которые нужно сохранить
char_preserve = ["й", "ё", "Ё"]
#знаки, на которые мы будем их менять
placeholders = ["@", "#", "%"]

In [7]:
def preprocess(text):
    text = text.lower()
    # Removing punctuations, numbers, and newline characters
    temp = remove_accents(text)
    res = replace_characters(placeholders, char_preserve, text)
    text = text.replace('\xa0', ' ')
    text = text.replace('\n\n', '\n')
    text = text.replace('()', '')
    text = text.replace('а́', 'а')
    return text

Посмотрим на результат препроцессинга текста, по первым 333 символам из статьи

In [10]:
preprocess(wiki_data['train'][50]['text'][:333])

'москва  — столица россии, город федерального значения, административный центр центрального федерального округа и центр московской области, в состав которой не входит. мегаполис; крупнейший по численности населения город россии и её субъект —  человек , самый населённый из городов, полностью расположенных в европе, занимает 22-'

При обработке документов будем разделять их по предложениями, но по желанию можно делить и по словам

In [8]:
def split_text(text: str, n=2, character=" ") -> List[str]:
    text = preprocess(text)
    
    # if want to split by words
    # text = text.split(character)
    # return [character.join(text[i : i + n]).strip() for i in range(0, len(text), n)]
    
    all_sentences = sent_tokenize(text)
    return [' '.join(all_sentences[i : i + n]) for i in range(0, len(all_sentences), 2)]


def split_documents(documents: List[str]) -> list:
    texts = []
    for text in documents:
        if text is not None:
            for passage in split_text(text):
                texts.append(passage)
    
    return texts


def embed(text, model, tokenizer):
    encoded_input = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors='pt').to(model.device)
    with torch.no_grad():
        model_output = model(**encoded_input)
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = encoded_input['attention_mask'].unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [16]:
split_text(wiki_data['train'][50]['text'])

['москва  — столица россии, город федерального значения, административный центр центрального федерального округа и центр московской области, в состав которой не входит. мегаполис; крупнейший по численности населения город россии и её субъект —  человек , самый населённый из городов, полностью расположенных в европе, занимает 22-е место среди городов мира по численности населения.',
 'центр московской городской агломерации. самый крупный город европы по площади.',
 'археологические данные свидетельствуют о существовании городского поселения в районе будущего кремля с xi века. первое письменное упоминание о москве относится к 1147 году: история города насчитывает, по меньшей мере, .',
 'москва — историческая столица великого княжества московского, русского царства, российской империи (в 1728—1732 годах), советской россии и ссср. город-герой.',
 'в москве находятся федеральные органы государственной власти российской федерации (за исключением конституционного суда), посольства иностранных

В качестве модели-векторизатора лучше выбирать наиболее качественную модель, которая выдает sentence-эмбединг для текста 

In [9]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
model.cuda();

In [10]:
embed(split_text(preprocess(wiki_data['train'][50]['text'])),  model, tokenizer).shape

torch.Size([343, 768])

Подготовим индекс для поиска

In [11]:
wiki_subsample_documents = split_documents(wiki_subsample)
batch_size = 32
total_batches = len(wiki_subsample_documents) // batch_size + (0 if len(wiki_subsample_documents) % batch_size == 0 else 1)
total_batches

3906

In [12]:
base = list()
for i in tqdm(range(0, len(wiki_subsample_documents), batch_size), total=total_batches, desc="Processing Batches"):
    batch_texts = wiki_subsample_documents[i:i + batch_size]
    base.extend(embed(batch_texts, model, tokenizer))
len(base)

Processing Batches:   0%|          | 0/3906 [00:00<?, ?it/s]

124984

In [13]:
base[2].cpu().shape

torch.Size([768])

In [14]:
base = np.array([vector.cpu().numpy() for vector in base])

In [15]:
import faiss

index = faiss.IndexFlatL2(base.shape[1])
index.add(base)

In [38]:
k = 4
query = 'столица России'
xq = embed(query.lower(), model, tokenizer).cpu().numpy()
D, I = index.search(xq.reshape(1, 768), k)  # search
print(I)

[[29413 92217 83425 81573]]


In [39]:
D

array([[ 88.50445 , 110.13212 , 111.28324 , 111.624535]], dtype=float32)

In [40]:
for ind in I[0]:
    print(wiki_subsample_documents[ind])
    print("*" * 80)

московский округ пво
пво российской федерации
********************************************************************************
волга – гордость россии / п.а. вышкинд.
********************************************************************************
примечания
ссылки 
 официальный сайт mazda в россии
********************************************************************************
113.
дирижёры россии
********************************************************************************


Обновим функцию, которая дает ответ на поддержку добавления полезного контекста

In [19]:
def get_completion(query: str, model, tokenizer, use_rag=True, context="") -> str:
  device = "cuda"

  prompt_template = """
  Ниже инструкция, которая описывает задачу. Напиши подробный ответ.
  ### Вопрос:
  {query}
  """
  if use_rag:
    prompt_template += f"""
    ### Релеватный контекст:
    {context}
    """
  prompt_template +=  "\n ### Ответ:"
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)


  generated_ids = model.generate(**model_inputs, max_new_tokens=200, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
  return (decoded[0])

In [23]:
del model
torch.cuda.empty_cache()

In [26]:
model_id = "TheBloke/Llama-2-7B-Chat-fp16"

tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
# model.cuda();

# model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")
llm_model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
llm_tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [42]:
query = "столица России"

xq = embed(query.lower(), model, tokenizer).cpu().numpy()
D, I = index.search(xq.reshape(1, 768), k)
context = wiki_subsample_documents[I[0][0]]

result = get_completion(query=query, model=llm_model, tokenizer=llm_tokenizer, use_rag=True, context=context)
print(result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



  Ниже инструкция, которая описывает задачу. Напиши подробный ответ.
  ### Вопрос:
  столица России
  
    ### Релеватный контекст:
    московский округ пво
пво российской федерации
    
 ### Ответ:0.1. Moscow is the capital of Russia.
0.2. Moscow is the largest city in Russia and the center of the Moscow Region.
0.3. Moscow is located in the eastern part of European Russia, on the banks of the Moscow River.
0.4. Moscow has a population of over 12 million people, making it one of the most populous cities in Europe.
0.5. Moscow is a major cultural, scientific, and educational center, with over 100 institutions of higher education.
0.6. Moscow is home to many famous landmarks, including the Kremlin, Red Square, and St. Basil's Cathedral.
0.7. Moscow has a complex history, having been the capital of various empires and states throughout the centuries, including the Russian Empire and the Soviet Union.
0.8. Moscow is a major transportation hub, with two international airports and a well-d

Как можете заметить модель теперь дает более качественный ответ без наллюцинаций.  
Чтобы улучшить эту систему вы можете добавить дообучение модели при использовании контекстного  ответа, это повысит качество модели

### Fine-Tune LLM

Во второй части занятия попробуем дообучить LLM на данных из задачи text-style transfer с использованием удобной обертки от huggingface 

In [8]:
# !pip install trl==0.4.7

In [3]:
from trl import SFTTrainer
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel
from transformers import TrainingArguments, pipeline
from datasets import Dataset

In [4]:
# QLoRA parameters

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

In [5]:
# bitsandbytes parameters

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [6]:
# TrainingArguments parameters

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = 500  # -1 for  full  dataset

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 100

# Log every X updates steps
logging_steps = 100

Обратите внимание, что мы задачи совсем небольшое количество шагов обучения. Это сделано для экономии времени обучения 

In [7]:
# SFT parameters

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [10]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

В качестве исходной модели опять возмем инструктивную версию, потому что у нас недостаточно данных, для полноценного обучения модели с предобученного чекпоинта

In [11]:
model_id = "TheBloke/Llama-2-7B-Chat-fp16"

llm_model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map=device_map)
llm_model.config.use_cache = False
llm_model.config.pretraining_tp = 1

llm_tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True, trust_remote_code=True, use_fast=False)
llm_tokenizer.add_special_tokens({'pad_token': '[PAD]'})
llm_tokenizer.pad_token = llm_tokenizer.eos_token
llm_tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

In [12]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [13]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

In [15]:
def get_completion(query, model, tokenizer):
    prompt = 'Ниже дан негативный комментарий, перепиши его в нейтральной форме\n\n'
    prompt += f'### Негативный комментарий:\n{query}\n\n'
    prompt += f'### Ответ:\n'
    encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
    model_inputs = encodeds.to(model.device)
    with torch.no_grad():
        generated_ids = model.generate(**model_inputs, max_new_tokens=200, do_sample=True, pad_token_id=tokenizer.eos_token_id)
    decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    return decoded[0]

Скорее всего модель ответит  нам  что-то непонятное, так как запрос на  русском языке, с которым она плохо дружит

In [15]:
result = get_completion(query="Что бля говорить когда не о чем говорить?", model=llm_model, tokenizer=llm_tokenizer)
print(result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Ниже дан негативный комментарий, перепиши его в нейтральной форме

### Негативный комментарий:
Что бля говорить когда не о чем говорить?

### Ответ:

Некоторые люди могут использовать язык для выражения своих мыслей и чувств, даже когда нет никакой точной темы для разговора. Это может быть связано с тем, что они хотят сказать что-то, но не знают, как выразить это в правильном формате. В таких случаях они могут использовать негативные комментарии, чтобы выразить свое разочарование или утомление.

Вот пример нейтрального ответа на негативный комментарий:
"Извините, но я не понимаю, что вы хотите сказать. Можете ли вы уточнить?"


In [16]:
data_path = '../3_week/data/input/train.tsv'

df = pd.read_csv(data_path, sep='\t')
df = df.fillna('')

df_train_toxic = []
df_train_neutral = []

for index, row in df.iterrows():
    references = row[['neutral_comment1', 'neutral_comment2', 'neutral_comment3']].tolist()
    
    for reference in references:
        if len(reference) > 0:
            df_train_toxic.append(row['toxic_comment'])
            df_train_neutral.append(reference)
        else:
            break

df = pd.DataFrame({
    'toxic_comment': df_train_toxic,
    'neutral_comment': df_train_neutral
})

df = shuffle(df)

df.sample(5)

toxic_comment  \
3672  урод ! сам не служил поэтому и не знает что та...   
1016   Ага и вы в ней чужие. Уезжайте, вам тут не рады.   
6688        Блять зачем она его позвала((((бля я боюсь(   
6935  с этой поломанной рукой ну вообще пиздец не уд...   
6973  я вообще-то там недавно сижу и у меня 11 фоток...   

                                        neutral_comment  
3672  Сам не служил поэтому не знает, что такое погоны!  
1016     Ага и вы в ней чужие. Уезжайте вам тут не рады  
6688                  Зачем она его позвала((( я боюсь(  
6935                        Со сломанной рукой неудобно  
6973  Я вообще-то там недавно сижу и у меня одиннадц...

In [17]:
def generate_prompt(data_point):
    # Samples with additional context into.
    if data_point['toxic_comment']:
        text = 'Ниже дан негативный комментарий, перепиши его в нейтральной форме\n\n'
        text += f'### Негативный комментарий:\n{data_point["toxic_comment"]}\n\n'
        text += f'### Ответ:\n{data_point["neutral_comment"]}'
    return text

# add the "prompt" column in the dataset
df["prompt"] = df.apply(generate_prompt, axis=1)

In [18]:
dataset = Dataset.from_pandas(df)

In [19]:
dataset = dataset.map(lambda samples: llm_tokenizer(samples["prompt"]), batched=True)

Map:   0%|          | 0/11090 [00:00<?, ? examples/s]

In [20]:
data = dataset.train_test_split(test_size=0.1)
train_data = data["train"]
test_data = data["test"]

In [21]:
trainer = SFTTrainer(
    model=llm_model,
    train_dataset=train_data,
    eval_dataset=test_data,
    peft_config=peft_config,
    dataset_text_field="prompt",
    max_seq_length=max_seq_length,
    tokenizer=llm_tokenizer,
    args=training_arguments,
    packing=packing,
)

/home/snv/Projects/courses/SF_generation_NLP/.venv/lib/python3.10/site-packages/peft/utils/other.py:143: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/home/snv/Projects/courses/SF_generation_NLP/.venv/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/9981 [00:00<?, ? examples/s]

Map:   0%|          | 0/1109 [00:00<?, ? examples/s]

/home/snv/Projects/courses/SF_generation_NLP/.venv/lib/python3.10/site-packages/accelerate/accelerator.py:437: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [22]:
llm_model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()
trainer.model.save_pretrained('./llama_2_trained')

Не забывайте включать кеширование в модели, на этапе inference

In [23]:
llm_model.config.use_cache = True

In [24]:
result = get_completion(query="Что бля говорить когда не о чем говорить?", model=llm_model, tokenizer=llm_tokenizer)
print(result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/home/snv/Projects/courses/SF_generation_NLP/.venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


Ниже дан негативный комментарий, перепиши его в нейтральной форме

### Негативный комментарий:
Что бля говорить когда не о чем говорить?

### Ответ:

Что говорить когда не о чем говорить?

### Ответ:
Что говорить когда не о чем говорить?

### Ответ:
Что говорить когда не о чем говорить?

### Ответ:
Что говорить когда не о чем говорить?

### Ответ:
Что говорить когда не о чем говорить?

### Ответ:
Что говорить когда не о чем говорить?

### Ответ:
Что говорить когда не о чем говорить?

### Ответ:
Что говорить когда не о чем говорить?

### Ответ:
Что говорить когда не о чем говорить?

### Ответ:
Что говорить когда не о чем гово


Модель теперь стала отвечать как надо,однако она зацикливается, что может свидетельствовать, что она недоучена

Чтобы использовать обученный адаптер вместе с моделью можно выполнить следующий код

In [13]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, './llama_2_trained')
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [17]:
result = get_completion(query="Что бля говорить когда не о чем говорить?", model=model, tokenizer=tokenizer)
print(result)

Ниже дан негативный комментарий, перепиши его в нейтральной форме

### Негативный комментарий:
Что бля говорить когда не о чем говорить?

### Ответ:
Что говорить когда не о чем говорить?

### Ответ:
Что говорить когда не о чем говорить?

### Ответ:
Что говорить когда не о чем говорить?

### Ответ:
Что говорить когда не о чем говорить?

### Ответ:
Что говорить когда не о чем говорить?

### Ответ:
Что говорить когда не о чем говорить?

### Ответ:
Что говорить когда не о чем говорить?

### Ответ:
Что говорить когда не о чем говорить?

### Ответ:
Что говорить когда не о чем говорить?

### Ответ:
Что говорить когда не о чем говори


Результаты совпадают, значит адаптер нормально обучился и сохранился и мы может эффективно теперь использовать его в дальнейших задачах

# Готово!